In [1]:
import cv2
import xml.etree.ElementTree as ET
from os import listdir

excluded = ["text"]

In [2]:
'''
Argument: Labels path (.xml)
Returns: List of xml files
'''
def list_xml_files(path=None):
    if path == None:
        print("Nenhuma pasta foi especificada.")
        return 0

    xml_files = []
    files = [f for f in listdir(path)]
    for f in files:
        if f[len(f)-4:] == ".xml":
            xml_files.append(f)

    return xml_files

In [3]:
'''
Argument: Images path (.jpg)
Returns: List of images name
'''
def list_images_name(path=None):
    if path == None:
        print("Nenhuma pasta foi especificada.")
        return 0

    images = []
    files = [f for f in listdir(path)]
    for f in files:
        if f[len(f)-4:] == ".jpg":
            images.append(f[:-4])

    return images

In [4]:
'''
Argument: xml file
Returns: Set with all the components of this file
'''
def normalization_v1(pascal_labels_path, xml, dic):
    tree = ET.parse(pascal_labels_path+xml)
    root = tree.getroot()

    components = []

    for child in root.findall("object"):
        components.append(child[0].text)

    # verifica: se tem aspas -> pega o que tem dentro
    # se não for, verifica se é connector Port
    # se não for, pega a primeira palavra
    for i in range(len(components)):
        quotes = components[i].split('"')
        if len(quotes)>=3 and quotes[0]=="connector " and quotes[1].split()[0]=="Port":
            print(components[i])
            components[i] = "connectorPort"
        elif len(quotes)>=3 and components[i][0]=='"':
            components[i] = quotes[1]
        else:
            components[i] = components[i].split()[0]
        
        if components[i] not in dic:
            dic[components[i]] = 1
        else:
            dic[components[i]] += 1
    
    return dic

In [5]:
'''
Argument: xml file
Returns: Set with all the components of this file
'''
def normalization_v2(pascal_labels_path, xml, dic):
    tree = ET.parse(pascal_labels_path+xml)
    root = tree.getroot()

    components = []

    for child in root.findall("object"):
        components.append(child[0].text)

    # verifica se é connector Port
    # se não, pega a primeira palavra
    for i in range(len(components)):
        words = components[i].split()
        if len(words)>=2:
            if words[0]+words[1]=='connector"Port':
                print(components[i])
                components[i] = "connectorPort"
            else:
                if '"' in words[0]:
                    components[i] = words[0].split('"')[1]
                else:
                    components[i] = words[0]
        else:
            components[i] = words[0]
            
        if components[i] not in dic:
            dic[components[i]] = 1
        else:
            dic[components[i]] += 1

    # print(components)
    
    return dic

In [6]:
'''
Argument: List of xml files
Does: A classes.txt file with all components of all xml_files
'''
def normalization(pascal_labels_path, xml_files):
    # create a empty set
    components = set()
    dic = {}
    for xml in xml_files:
        dic = normalization_v2(pascal_labels_path, xml, dic)
    
    classes = open("classes.txt", "w")
    for component in sorted(dic):
        classes.write(component+"\n")

    classes.close()
    return dic

In [7]:
xml_files = list_xml_files("/home/jacq/Documentos/Datasets/pcb_pascal/labels/")

In [8]:
dic_components = normalization("/home/jacq/Documentos/Datasets/pcb_pascal/labels/", xml_files)

connector "Port 4"
connector "Port 3"
connector	"Port 2"
connector "Port 1"


In [9]:
sorted_dic = {}
for item in sorted(dic_components):
    sorted_dic[item] = dic_components[item] 

In [10]:
sorted_dic

{'battery': 1,
 'button': 85,
 'buzzer': 1,
 'capacitor': 2534,
 'clock': 37,
 'component': 995,
 'connector': 617,
 'connectorPort': 4,
 'diode': 72,
 'display': 5,
 'electrolytic': 246,
 'emi': 51,
 'ferrite': 30,
 'fuse': 7,
 'heatsink': 4,
 'ic': 386,
 'inductor': 69,
 'jumper': 85,
 'led': 214,
 'pads': 332,
 'pins': 317,
 'potentiometer': 7,
 'resistor': 2191,
 'switch': 58,
 'test': 292,
 'text': 9190,
 'transformer': 1,
 'transistor': 84,
 'unknown': 281,
 'zener': 5}

In [52]:
# problems: connector Port, connector e components

In [53]:
# problem: pq treinar components?

In [54]:
'''
Argument: A classes.txt file
Does: A labels.txt file with all components that are not excluded or repeated
Returns: A components dictionary in which components have a respective number
'''
def components_dictionary(classes_file, excluded, repeated):
    components_dict = {}

    classes = open(classes_file, "r")
    labels = open("labels.txt", "w")
    component_number = 0
    for component in classes:
        if component[:-1] not in excluded:
            if component[:-1] in repeated:
                component_number -= 1
                components_dict[component[:-1]] = component_number
            else:
                components_dict[component[:-1]] = component_number
                labels.write(component)
            component_number += 1

    labels.close()
    classes.close()

    return components_dict

In [55]:
def pascal_to_yolo_v1(yolo_images_path, yolo_labels_path, pascal_labels_path, images_name, components_dict):
    for name in images_name:
        image = cv2.imread(yolo_images_path+name+".jpg")
        yolo = open(yolo_labels_path+name+".txt", 'w')

        tree = ET.parse(pascal_labels_path+name+".xml")
        root = tree.getroot()

        for child in root.findall("object"):
            component_name = child[0].text

            quotes = component_name.split('"')
            if len(quotes)>=3 and quotes[0]=="connector " and quotes[1].split()[0]=="Port":
                component_name = "connectorPort"
                # print(component_name)
            elif len(quotes)>=3 and component_name[0]=='"':
                component_name = quotes[1]
            else:
                component_name = component_name.split()[0]

            xmin = int(child[4][0].text)
            ymin = int(child[4][1].text)
            xmax = int(child[4][2].text)
            ymax = int(child[4][3].text)

            #print(image.shape)
            px = image.shape[1]
            py = image.shape[0]

            # cálculo pro quadradinho ser igual o do yolo
            # ponto no centro, altura e largura, dividos pelo tamanho da imagem
            X = ((xmax+xmin)/2)/px
            Y = ((ymax+ymin)/2)/py
            W = (xmax-xmin)/px
            H = (ymax-ymin)/py

            if component_name not in excluded:
                yolo.write(str(components_dict[component_name])+" "+str(X)+" "+str(Y)+" "+str(W)+" "+str(H)+"\n")

        yolo.close()

In [56]:
'''
Arguments: List of images name, dictionary with components and its numbers
Does: A .txt file to every image in yolo format
'''
def pascal_to_yolo_v2(yolo_images_path, yolo_labels_path, pascal_labels_path, images_name, components_dict):
    for name in images_name:
        image = cv2.imread(yolo_images_path+name+".jpg")
        yolo = open(yolo_labels_path+name+".txt", 'w')

        tree = ET.parse(pascal_labels_path+name+".xml")
        root = tree.getroot()

        for child in root.findall("object"):
            component_name = child[0].text

            words = component_name.split()
            if len(words)>=2:
                if words[0]+words[1]=='connector"Port':
                    print(component_name)
                    component_name = "connectorPort"
                else:
                    if '"' in words[0]:
                        component_name = words[0].split('"')[1]
                    else:
                        component_name = words[0]
            else:
                component_name = words[0]

            xmin = int(child[4][0].text)
            ymin = int(child[4][1].text)
            xmax = int(child[4][2].text)
            ymax = int(child[4][3].text)

            #print(image.shape)
            px = image.shape[1]
            py = image.shape[0]

            # cálculo pro quadradinho ser igual o do yolo
            # ponto no centro, altura e largura, dividos pelo tamanho da imagem
            X = ((xmax+xmin)/2)/px
            Y = ((ymax+ymin)/2)/py
            W = (xmax-xmin)/px
            H = (ymax-ymin)/py

            if component_name not in excluded:
                yolo.write(str(components_dict[component_name])+" "+str(X)+" "+str(Y)+" "+str(W)+" "+str(H)+"\n")

        yolo.close()

In [57]:
components_dict = components_dictionary("classes.txt",["text"], [])

In [58]:
components_dict

{'battery': 0,
 'button': 1,
 'buzzer': 2,
 'capacitor': 3,
 'clock': 4,
 'component': 5,
 'connector': 6,
 'connectorPort': 7,
 'diode': 8,
 'display': 9,
 'electrolytic': 10,
 'emi': 11,
 'ferrite': 12,
 'fuse': 13,
 'heatsink': 14,
 'ic': 15,
 'inductor': 16,
 'jumper': 17,
 'led': 18,
 'pads': 19,
 'pins': 20,
 'potentiometer': 21,
 'resistor': 22,
 'switch': 23,
 'test': 24,
 'transformer': 25,
 'transistor': 26,
 'unknown': 27,
 'zener': 28}

In [59]:
images = list_images_name("/home/jacq/Documentos/Datasets/pcb_pascal/images/")

In [60]:
len(images)

47

In [61]:
pascal_labels_path = "/home/jacq/Documentos/Datasets/pcb_pascal/labels/"
yolo_images_path = "/home/jacq/Documentos/Datasets/pcb_yolo/images/"
yolo_labels_path = "/home/jacq/Documentos/Datasets/pcb_yolo/labels/"
pascal_to_yolo_labelImg(yolo_images_path, yolo_labels_path, pascal_labels_path, images, components_dict)

connector "Port 4"
connector "Port 3"
connector	"Port 2"
connector "Port 1"


In [85]:
def stats_database(file, components_dict):
    file = open(file)
    
    stats_dict = {}
    
    for item in components_dict:
        stats_dict[str(components_dict[item])] = 0
        
#     print(stats_dict)
    
    for line in file:
        line = line.split()
        line = line[1:]
        for component in line:
            component = component.split(",")
            stats_dict[str(component[4])]+=1
            
    for item in components_dict:
        components_dict[item] = stats_dict[str(components_dict[item])] 
            
    return components_dict

In [86]:
components_dict

{'battery': 0,
 'button': 1,
 'buzzer': 2,
 'capacitor': 3,
 'clock': 4,
 'component': 5,
 'connector': 6,
 'connectorPort': 7,
 'diode': 8,
 'display': 9,
 'electrolytic': 10,
 'emi': 11,
 'ferrite': 12,
 'fuse': 13,
 'heatsink': 14,
 'ic': 15,
 'inductor': 16,
 'jumper': 17,
 'led': 18,
 'pads': 19,
 'pins': 20,
 'potentiometer': 21,
 'resistor': 22,
 'switch': 23,
 'test': 24,
 'transformer': 25,
 'transistor': 26,
 'unknown': 27,
 'zener': 28}

In [87]:
stats_database("train_augmented.txt", components_dict)

{'battery': 32,
 'button': 2720,
 'buzzer': 32,
 'capacitor': 81088,
 'clock': 1184,
 'component': 19776,
 'connector': 96,
 'connectorPort': 2304,
 'diode': 160,
 'display': 7872,
 'electrolytic': 1632,
 'emi': 960,
 'ferrite': 224,
 'fuse': 128,
 'heatsink': 12352,
 'ic': 2208,
 'inductor': 2720,
 'jumper': 6848,
 'led': 10624,
 'pads': 10144,
 'pins': 224,
 'potentiometer': 70112,
 'resistor': 1856,
 'switch': 9344,
 'test': 32,
 'transformer': 2688,
 'transistor': 8992,
 'unknown': 160,
 'zener': 0}